In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)
 
# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Downloads\NEMESIS\unit_test
切换后的目录： c:\Users\zhaoxs3\Downloads\NEMESIS


In [2]:
from devlib.market.curves.ccs_curve import CnyUsdCcsCurve
from devlib.market.curves.overnight_index_curves import Sofr
import QuantLib as ql
import pandas as pd

In [3]:
today = ql.Date(5, 8, 2024)
calendar = ql.JointCalendar(ql.China(ql.China.IB), ql.UnitedStates(ql.UnitedStates.FederalReserve), ql.JoinHolidays)
convention = ql.ModifiedFollowing
valuation_date = today

ql.Settings.instance().evaluationDate = valuation_date

############
pair = 'USDCNY'
fx_spot = 7.1396

# 构建usd discount curve
mkt_file_path = './unit_test/data/sofr_curve_data_20240805.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
sofr_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

# 构建cnh discount curve (cicc ccs curve version)
mkt_file_path =  './unit_test/data/USDCNY_ccs_curve_data_20240805.xlsx'
fx_swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='fx')
ccs_mkt_data = pd.read_excel(mkt_file_path, sheet_name='ccs')

cny_discount_curve = CnyUsdCcsCurve(today, collateral_index_curve=sofr_curve, collateral_discount_curve=sofr_curve,
                                    ccs_mkt_data=ccs_mkt_data, fx_swap_mkt_data=fx_swap_mkt_data)

In [4]:
from nemesis.products.rates import *
from nemesis.utils import *
from nemesis.products.rates.ccs_curve import CCSCurve
from nemesis.products.fx.fx_forward_curve import FXForward

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [5]:
value_dt = Date(5,8,2024)
spot_days = 2

cal_type = JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES])
calendar = Calendar(cal_type)
settle_dt = calendar.add_business_days(value_dt, spot_days)

In [6]:
fx_swap_mkt_data_dict = fx_swap_mkt_data.to_dict(orient='records')

In [7]:
spot = fx_swap_mkt_data_dict[0].get("Spread")

In [8]:
sofr = QLCurve(value_dt, sofr_curve, dc_type=DayCountTypes.ACT_360, interp_type=InterpTypes.LINEAR_ZERO_RATES)

In [9]:
fx_forwards = [FXForward(settle_dt, record.get("Tenor"), spot, record.get("Spread"), dc_type=DayCountTypes.ACT_365F, cal_type=cal_type, bd_type=BusDayAdjustTypes.FOLLOWING) for record in fx_swap_mkt_data_dict[1:]]

In [10]:
notional = 1

fixed_type = SwapTypes.PAY
fixed_freq = FrequencyTypes.QUARTERLY
fixed_dc_type = DayCountTypes.ACT_365F

spread = 0
float_freq = FrequencyTypes.QUARTERLY
float_dc_type = DayCountTypes.ACT_365F

bd_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
dg_type = DateGenRuleTypes.BACKWARD

In [11]:
swaps = [
    OIS(
        effective_dt = settle_dt,
        term_dt_or_tenor = tenor,
        fixed_leg_type = fixed_type,
        fixed_cpn = rate,
        fixed_freq_type = fixed_freq,
        fixed_dc_type = fixed_dc_type,
        notional = notional,
        payment_lag = 2,
        float_spread = spread,
        float_freq_type = float_freq,
        float_dc_type = float_dc_type,
        cal_type = cal_type,
        bd_type = bd_type,
        dg_type = dg_type
    )
    for tenor, rate in zip(ccs_mkt_data["Tenor"], ccs_mkt_data["Rate"]) 
]

In [12]:
mat_dates = [fwd.maturity_dt for fwd in fx_forwards] + [calendar.add_business_days(swap.maturity_dt, 2) for swap in swaps]

In [13]:
curve = CCSCurve(value_dt=value_dt, fx_forwards=fx_forwards, ois_swaps=swaps, foreign_curve=sofr, interp_type=InterpTypes.LINEAR_ZERO_RATES)

In [14]:
curve.df(mat_dates, day_count=DayCountTypes.ACT_365F)

array([0.99989119, 0.99940517, 0.99879558, 0.99795676, 0.99616876,
       0.99496158, 0.99381261, 0.99018125, 0.98799529, 0.97445362,
       0.95701762, 0.94714215, 0.89981936, 0.82192016])

In [15]:
curve.print_table(mat_dates)

,Date,ZR,DF
0,2024-08-14,0.44131,0.999891
1,2024-09-09,0.62050,0.999405
2,2024-10-08,0.68731,0.998796
3,2024-11-07,0.79420,0.997957
4,2025-02-07,0.75327,0.996169
5,2025-05-07,0.67043,0.994962
6,2025-08-07,0.61728,0.993813
7,2026-02-09,0.65128,0.990181
8,2026-08-11,0.59894,0.987995
9,2027-08-11,0.85791,0.974454
